# Config

In [43]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import t
import numpy as np

# Problemas
## Exercício 05
### Item a

In [38]:
# Criando o DataFrame
data = {
    "Indivíduo": list(range(1, 21)),
    "Y": [96, 92, 106, 100, 98, 104, 110, 101, 116, 106,
          109, 100, 112, 105, 118, 108, 113, 112, 127, 117],
    "W": ["H", "M", "H", "M", "M", "H", "H", "M", "M", "H",
          "H", "M", "M", "M", "H", "H", "M", "M", "H", "H"],
    "X": [20, 20, 20, 20, 25, 25, 25, 25, 30, 30,
          30, 30, 35, 35, 35, 35, 40, 40, 40, 40],
    "Z": [90, 100, 80, 90, 100, 90, 80, 90, 70, 90,
          90, 80, 90, 80, 70, 90, 90, 90, 60, 80]
}

df = pd.DataFrame(data)

# Exibindo o DataFrame
df.head()

,Indivíduo,Y,W,X,Z
0,1,96,H,20,90
1,2,92,M,20,100
2,3,106,H,20,80
3,4,100,M,20,90
4,5,98,M,25,100


In [39]:
# Ajustando o modelo de regressão Z = a + bX
model = smf.ols("Z ~ X", data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Z   R-squared:                       0.159
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     3.408
Date:                Wed, 05 Mar 2025   Prob (F-statistic):             0.0814
Time:                        21:15:27   Log-Likelihood:                -72.183
No. Observations:                  20   AIC:                             148.4
Df Residuals:                      18   BIC:                             150.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    101.5000      9.182     11.054      0.000      82.209     120.791
X             -0.5500      0.298     -1.846      0.081      -1.176       0.076
==============================================================================
Omnibus:                        1.711   Durbin-Watson:                   2.460
Prob(Omnibus):                  0.425   Jarque-Bera (JB):                1.464
Skew:                          -0.572   Prob(JB):                        0.481
Kurtosis:                       2.333   Cond. No.                         134.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
# Gerando a tabela ANOVA
anova_table = sm.stats.anova_lm(model, typ=1)  # Tipo II ANOVA
print(anova_table)

            df  sum_sq  mean_sq         F    PR(>F)
X          1.0   302.5   302.50  3.408451  0.081375
Residual  18.0  1597.5    88.75       NaN       NaN


## Exercício 10

### Item a

In [41]:
intervalos = model.conf_int(alpha=0.05)

# Nomeando as colunas para facilitar a leitura
intervalos.columns = ["Limite Inferior", "Limite Superior"]

# Adicionando os nomes dos coeficientes
intervalos.index = ["Intercepto", "Coeficiente de X"]

intervalos

# Realizar validação manual...

,Limite Inferior,Limite Superior
Intercepto,82.208948,120.791052
Coeficiente de X,-1.175884,0.075884


In [51]:
# Valor da t-student com 18 graus de liberdade e 95% de confiança
t_critical = t.ppf(1 - 0.025, 18)

# -- Estimativa de sigma^2e
# Método 01: Usando a biblioteca
Se_1 = np.sqrt(model.mse_resid)
# Método 02: Usando a fórmula (SQRes / GLRes)
SQRes = model.ssr
GLRes = model.df_resid
Se_2 = np.sqrt(SQRes / GLRes)

print(Se_1 == Se_2)
Se = Se_1

# -- Raiz(1/Sxx)
Sxx = sum((df["X"] - df["X"].mean())**2)
raiz_Sxx = 1 / Sxx**(1/2)

# Estimativa do coeficiente de X
b_hat = model.params["X"]

# -- Intervalo de confiança
inferior = b_hat - t_critical * raiz_Sxx * Se
superior = b_hat + t_critical * raiz_Sxx * Se
print(inferior, superior)

b_hat, Sxx, Se

True
-1.1758843870487248 0.0758843870487258


(-0.5499999999999995, 1000.0, 9.420721840708387)

### Item c

In [56]:
-t_critical < (b_hat/Se) * (Sxx)**(1/2) < t_critical
# Tal que a estimativa não pertence a região crítica. Logo, a hipótese nula não é rejeitada.

True